## Retrieve crawled web pages (2009-present) via Commoncrawl/AWS/Athena

This process uses Kiara to access and retrieve web pages indexed by [Commoncrawl](https://commoncrawl.org/). Users can perform a SQL query on Commoncrawl stored documents to retrieve the indexes of web pages. It is then possible to retrieve the corresponding web pages.<br>
The querying process may trigger billing by AWS since it uses Athena services to scan Commoncrawl stored data (5.00 USD per Terabytes, Oct. 2023).

In [61]:
from kiara.api import KiaraAPI
kiara = KiaraAPI.instance()

### AWS Credentials

In [2]:
aws_access_key_id = 'myawsaccesskey'
aws_secret_access_key = 'myawssecretaccesskey'
aws_s3_bucket = 'mys3bucket'
# the name of the database to create
db_name = 'cc_test'
# the name of the table to store query results
table_name = 'mossfon'

### Query set-up to scan crawled data files

In the following example, Athena will scan for Commoncrawl web pages for the domain name "mossfon.com" and limit the results to 10. Since no period is indicated, it will look at the whole Commoncrawl data from 2009 to now. See https://commoncrawl.org/blog/index-to-warc-files-and-urls-in-columnar-format for more information and more examples of queries.

In [4]:
query= f"""
SELECT *
FROM {db_name}.{table_name}
WHERE url_host_registered_domain = 'mossfon.com'
LIMIT 100
"""

### I. Execute Athena query to find content on Commoncrawl

In [64]:
! kiara operation explain onboard.run_cc_query


╭─ Operation: onboard.run_cc_query ────────────────────────────────────────────╮
│                                                                              │
│   Documentation   Execute a Common Crawl indexes query via Amazon Web        │
│                   Services (AWS) and Athena.                                 │
│                                                                              │
│                   This process requires an AWS account and an S3 bucket.     │
│                   It may trigger some fees billed by AWS.                    │
│                   Additional information on the process followed available   │
│                   at:                                                        │
│                   https://commoncrawl.org/2018/03/index-to-warc-files-and…   │
│                                                                              │
│   Inputs                                                                     │
│                     field

In [65]:
inputs = {
    "aws_access_key_id": aws_access_key_id,
    "aws_secret_access_key": aws_secret_access_key,
    "aws_s3_bucket": aws_s3_bucket,
    "query": query,
    "db_name": db_name,
    "table_name": table_name,
 }

In [66]:
query_id = kiara.run_job('onboard.run_cc_query', inputs=inputs)

In [67]:
query_id

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field         value                                                                                                                    │
│  ────────────────────────────────────────────────────                                                                                    │
│   cc_query_id   47f2f1df-45ea-4901-8334-e4d11fba87b2                                                                                     │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [68]:
query_id['cc_query_id'].data

'47f2f1df-45ea-4901-8334-e4d11fba87b2'

### II. Get query execution status

Before retrieving the results, check that the query execution process is finished.

In [69]:
! kiara operation explain onboard.get_cc_query_status


╭─ Operation: onboard.get_cc_query_status ─────────────────────────────────────╮
│                                                                              │
│   Documentation   Get the status of a Common Crawl indexes query.            │
│                                                                              │
│   Inputs                                                                     │
│                     field                                                    │
│                     name        type     descrip…   Required   Default       │
│                    ──────────────────────────────────────────────────────    │
│                     cc_query_   string   AWS/Ath…   yes        -- no         │
│                     id                   query                 default       │
│                                          id.                   --            │
│                     aws_acces   string   The AWS    yes        -- no         │
│                     s_key

In [70]:
# TODO: check why error if the value is entered directly as a string
# "cc_query_id": '9b18ffd9-98b8-482a-9fdd-803fd7636b91'

inputs = {
    "aws_access_key_id": aws_access_key_id,
    "aws_secret_access_key": aws_secret_access_key,
    # this doesn't work in kiara at the moment "cc_query_id": '9b18ffd9-98b8-482a-9fdd-803fd7636b91',
    "cc_query_id": query_id['cc_query_id'], 
 }

In [109]:
# wait until the output of this cell displays "state: succeeded" before processing the notebook further
query_status = kiara.run_job('onboard.get_cc_query_status', inputs=inputs)
query_status.get_value_data('cc_query_status').dict_data['QueryExecution']['Status']

{'State': 'SUCCEEDED',
 'SubmissionDateTime': datetime.datetime(2023, 11, 8, 19, 20, 0, 850000, tzinfo=tzlocal()),
 'CompletionDateTime': datetime.datetime(2023, 11, 8, 19, 22, 41, 821000, tzinfo=tzlocal())}

### III. Retrieve query results

The result of the query (if any) contains the indexes necessary to access the related web pages content.

In [110]:
! kiara operation explain onboard.get_cc_query_result


╭─ Operation: onboard.get_cc_query_result ─────────────────────────────────────╮
│                                                                              │
│   Documentation   Get the result of a Common Crawl documents indexes         │
│                   query.                                                     │
│                                                                              │
│   Inputs                                                                     │
│                     field                                                    │
│                     name        type     descrip…   Required   Default       │
│                    ──────────────────────────────────────────────────────    │
│                     cc_query_   string   AWS/Ath…   yes        -- no         │
│                     id                   query                 default       │
│                                          id.                   --            │
│                     aws_a

In [111]:
query_result = kiara.run_job('onboard.get_cc_query_result', inputs=inputs)

### IV. Get web pages content

With the exact location in Commoncrawl compressed storage files, it is now possible to retrieve the content of the web pages.

In [112]:
! kiara operation explain onboard.get_cc_pages


╭─ Operation: onboard.get_cc_pages ────────────────────────────────────────────╮
│                                                                              │
│   Documentation   Get the web pages from common crawl indexes.               │
│                                                                              │
│   Inputs                                                                     │
│                     field                                                    │
│                     name        type   descript…   Required   Default        │
│                    ──────────────────────────────────────────────────────    │
│                     cc_query_   dict   Web pages   yes        -- no          │
│                     result             retrieved              default --     │
│                                        from                                  │
│                                        commoncr…                             │
│                          

In [113]:
inputs = {
    "cc_query_result": query_result['cc_query_result'],
 }

In [114]:
query_pages = kiara.run_job('onboard.get_cc_pages', inputs=inputs)

In [115]:
cc_filename = query_pages['cc_query_pages'].data.dict_data['cc_filename']

In [116]:
web_page = query_pages['cc_query_pages'].data.dict_data['web_page']

In [117]:
status = query_pages['cc_query_pages'].data.dict_data['status']

In [118]:
# process below is temporary just to have a quick look at the output
# I didn't put pandas as a dependency

In [119]:
import pandas as pd

In [120]:
df = pd.DataFrame({'cc_filename': cc_filename, 'web_page': web_page, 'status': status})

In [121]:
df['word_count'] = df['web_page'].str.len()

In [122]:
res_preview = df[df['status'] == 'success'].sort_values(by=['word_count'], ascending=False).copy()

In [123]:
res_preview.head(30)

,cc_filename,web_page,status,word_count
205,crawl-data/CC-MAIN-2017-13/segments/1490218186...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,80360.0
373,crawl-data/CC-MAIN-2018-13/segments/1521257647...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,80225.0
406,crawl-data/CC-MAIN-2019-43/segments/1570987795...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,68455.0
344,crawl-data/CC-MAIN-2019-47/segments/1573496668...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,68428.0
799,crawl-data/CC-MAIN-2013-48/segments/1386164346...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,66901.0
797,crawl-data/CC-MAIN-2013-48/segments/1386163052...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,61802.0
798,crawl-data/CC-MAIN-2013-48/segments/1386164019...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,61005.0
637,crawl-data/CC-MAIN-2013-20/segments/1368696382...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,58859.0
636,crawl-data/CC-MAIN-2013-20/segments/1368699776...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,53833.0
635,crawl-data/CC-MAIN-2013-20/segments/1368699924...,WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: ...,success,53586.0


In [124]:
res_preview.to_csv('web_pages.csv')